In this notebook I intend to import the annotated data and establish a baseline performance of the non-fine-tuned davinci-instruct model <br>
prerequistes:
- A wroking prompt
- PoC that davinci works better than the other models<br><br>
Steps after:
- finetune model

next up: now that I've got a reasonable prompt for the large models, lets compare against the small ones and write up a report for thales - by tonight

In [2]:
import pandas as pd
import pickle as pkl
import openai
from sklearn.metrics import confusion_matrix
import time
import json

# import large file
with open('annotated_sample.pkl', 'rb') as f:
    pre_df = pkl.load(f)

# shorten to not murder my computer
# df = pre_df.iloc[:50]
#del pre_df

# retrieve API key
import os
api_key = os.environ.get('OPENAI_API_KEY')
# print(api_key)

In [13]:
pre_df[(pre_df["has_disclosures"]==True)&(pre_df["label"]==True)]

,username,caption,caption_hashtags,tagged_users,tagged_users_full,likes,comments,is_ad,caption_is_edited,location,...,dt_ordinal,dt_year_mon,caption_text_symbols,has_sponsored_keywords,has_disclosures,keyword_tfidf,keyword_keybert,captions_task,caption_task,label
87072,weworewhat,You guys know I’m obsessed with blue light gla...,"[#thedewers, #ad]",[goodhabitskin],[Goodhabit],11169,85,False,True,"Westhampton Beach, New York",...,737530,2020/04,you guys know i m obsessed with blue light gla...,True,True,"[blue light, blue, light, it protects against,...","[exposure blue light, obsessed blue light, exp...",You guys know I’m obsessed with blue light gla...,You guys know I’m obsessed with blue light gla...,True
87105,weworewhat,Moving my body during this time has not only h...,"[#fpmovement, #ad]",[],[],27161,259,False,True,"Westhampton Beach, New York",...,737517,2020/04,moving my body during this time has not only h...,True,True,"[workouts, days that we, kept us active, activ...","[live workouts, free live workouts, workouts g...",Moving my body during this time has not only h...,Moving my body during this time has not only h...,True
87181,weworewhat,new @shopweworewhat coverup in a print inspire...,"[#iwantklarna, #klarnapartner, #ad]",[klarna.usa],[Klarna US],18029,305,False,False,,...,737476,2020/02,new @shopweworewhat coverup in a print inspire...,True,True,"[vintage rug forget, by my bedroom, new coveru...","[new coverup print, rug forget shop, new cover...",new @shopweworewhat coverup in a print inspire...,new @shopweworewhat coverup in a print inspire...,True
87196,weworewhat,Another rainy day but my curated @gilt sale en...,"[#gotitongilt, #ad]",[gilt],[Gilt.com],19554,101,False,False,,...,737468,2020/02,another rainy day but my curated @gilt sale en...,True,True,"[curated sale ends, added few more, more piece...","[curated sale ends, rainy day curated, day cur...",Another rainy day but my curated @gilt sale en...,Another rainy day but my curated @gilt sale en...,True
87251,weworewhat,"Guys, I curated another sale on @gilt where yo...","[#gotitongilt, #ad]","[gilt, petersyn_clothing]","[Gilt.com, Petersyn]",17991,148,False,False,,...,737439,2020/01,guys i curated another sale on @gilt where you...,True,True,"[curated another sale, items tonight at, curat...","[luxury items tonight, luxury items, affordabl...","Guys, I curated another sale on @gilt where yo...","Guys, I curated another sale on @gilt where yo...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,michellesalasb,Good shoes take you to good places 😎😜 So much ...,"[#mgemiandiamo, #ad]","[fiat, angeloctaves, francisco.solana, mgemi]","[Fiat, ANGEL, S O L Λ N Λ, M.Gemi]",18749,39,False,True,"New York, New York",...,736931,2018/08,good shoes take you to good places so much fun...,True,True,"[spin last week, spin last, places so much, 12...","[fiat 124 spider, 124 spider, spider spin week...",Good shoes take you to good places 😎😜 So much ...,Good shoes take you to good places 😎😜 So much ...,True
811,michellesalasb,Femininity is a weapon of power💘Love it and em...,"[#IntimissimiMx, #Italianlingerie, #ad]",[intimissimiofficial],[Intimissimi],23036,201,False,True,"Los Angeles, California",...,736817,2018/05,femininity is a weapon of power love it and em...,True,True,"[it and embrace, femininity is weapon, power l...","[femininity weapon power, femininity weapon, f...",Femininity is a weapon of power💘Love it and em...,Femininity is a weapon of power💘Love it and em...,True
981,michellesalasb,"Find your family, love them hard. Our Family. ...","[#MYCALVINS, #ad]","[calvinklein, francisco.solana]","[CALVIN KLEIN, S O L Λ N Λ]",16614,137,False,False,"New York, New York",...,736717,2018/01,find your family love them hard our family #my...,True,True,"[hard our family, them hard our, find your fam...","[family love hard, love hard family, family lo...","Find your family, love them hard. Our Family. ...","Find your family, love t

In [3]:
# sample 50 ads, 50 non-ads
df = pd.concat([pre_df[pre_df.loc[:,"label"]==True].sample(30),
                pre_df[pre_df.loc[:,"label"]==False].sample(30)]).copy()
df.reset_index(drop=True,inplace=True)
del pre_df

In [4]:
df

,username,caption,caption_hashtags,tagged_users,tagged_users_full,likes,comments,is_ad,caption_is_edited,location,...,dt_ordinal,dt_year_mon,caption_text_symbols,has_sponsored_keywords,has_disclosures,keyword_tfidf,keyword_keybert,captions_task,caption_task,label
0,oliviadbowen,Eyeing up the suitcases 👀☀️🌊\nOutfit from @reb...,"[#RebelGal, #ad]",[rebelliousfashion],[Rebellious Fashion],95897,396,False,True,,...,737132,2019/03,eyeing up the suitcases outfit from @rebelliou...,True,True,"[20578, outfit from codes, top 20578 trousers,...","[eyeing suitcases outfit, suitcases outfit cod...",Eyeing up the suitcases 👀☀️🌊 Outfit from @rebe...,Eyeing up the suitcases 👀☀️🌊 Outfit from @rebe...,True
1,chrisjerichofozzy,Ready for #JerichoVsNaito2... #WrestleKingdom1...,"[#JerichoVsNaito2..., #WrestleKingdom13]",[],[],111731,979,False,False,東京ドームシティ Tokyo Dome City,...,737063,2019/01,ready for #jerichovsnaito2 #wrestlekingdom13 j...,False,False,"[by tights, by tights by, for jacket by, jacke...","[ready jacket tights, ready jacket, jacket tig...",Ready for #JerichoVsNaito2... #WrestleKingdom1...,Ready for #JerichoVsNaito2... #WrestleKingdom1...,True
2,swopes,Chicago has scooters thanks to @lyft! 💓Explori...,"[#DoYourCityRight, #ad]",[],[],3654,66,False,False,"Chicago, Illinois",...,737339,2019/10,chicago has scooters thanks to @lyft exploring...,True,True,"[has scooters thanks, has scooters, thanks to ...","[chicago scooters, chicago scooters thanks, sc...",Chicago has scooters thanks to @lyft! 💓Explori...,Chicago has scooters thanks to @lyft! 💓Explori...,True
3,zoesugg,Just received this lovely package from @lilype...,[],[],[],202717,243,False,False,,...,736762,2018/03,just received this lovely package from @lilype...,False,False,"[lily it, copy thank, female friendships, fema...","[beautiful book, beautiful book proud, lily be...",Just received this lovely package from @lilype...,Just received this lovely package from @lilype...,True
4,beberexha,My mood for the whole year. Just being myself ...,[],[standstudio.official],[STAND STUDIO],330400,1384,False,True,"Los Angeles, California",...,737430,2020/01,my mood for the whole year just being myself a...,False,False,"[living, year just being, want no longer, for ...","[mood year just, mood year, year just living, ...",My mood for the whole year. Just being myself ...,My mood for the whole year. Just being myself ...,True
5,ijustine,🙄 Snap the code on your @Pepsi bottles to unlo...,"[#PepsiSummer, #ad]",[],[],25218,67,False,False,,...,736527,2017/07,snap the code on your @pepsi bottles to unlock...,True,True,"[on your bottles, bottles to unlock, win prize...","[prizes awesome mobile, bottles unlock chance,...",🙄 Snap the code on your @Pepsi bottles to unlo...,🙄 Snap the code on your @Pepsi bottles to unlo...,True
6,glamdapper,👨‍👩‍👧 One of our favorite parts of 💐 Spring i...,"[#ad, #MarshallsSurprise, #MarshallsSurprise, ...","[marshalls, glamdapper]","[Marshalls, Rosanna & Winston | Couple]",8778,699,False,True,"Nassau County, New York",...,736789,2018/04,one of our favorite parts of spring is refresh...,True,True,"[de, unas de nuestra, unas de, de la, our]","[favoritas la primavera, spring looks includin...",👨‍👩‍👧 One of our favorite parts of 💐 Spring is...,👨‍👩‍👧 One of our favorite parts of 💐 Spring is...,True
7,iamcattsadler,While you’re stocking up on your essentials to...,[#Ad],"[vitalproteins, costco]","[Vital Proteins, Costco]",1234,26,False,False,,...,737500,2020/03,while you re stocking up on your essentials to...,True,True,"[new variety 35, marathon training swear, plen...","[35 count bag, essentials stay healthy, count ...",While you’re stocking up on your essentials to...,While you’re stocking up on your essentials to...,True
8,devourpower,Today we stopped into @fenianspub in #Jackson ...,"[#Jackson, #friedchicken, #DEVOURPOWER, #Visit...","[visitjacksonms, fenianspub, devour.friedchicken]","[The City with Soul | JXN, MS, Fenian's Pub, #...",12283,116,False,True,Fenian's Pub,...,

In [5]:
# Generate classifications for davinci only

completions ={"0":[],"1":[]} #{"text-ada-001":[], "text-babbage-001":[], "text-curie-001":[], "text-davinci-002":[]}
i=0
for x in range(1):#,"text-ada-001", "text-babbage-001", "text-curie-001"]:
    #i=0
    for txt in df.loc[:,"caption"]:
        i+=1
        if i%10==0: print(f"Counter at {i}")

        # timer to stay within my quota
        if i==29:
            print(f"sleep at {i}") 
            time.sleep(65)
        if i==59: time.sleep(65)
        if i==89: time.sleep(65)
        
        prompt = f"Might the following post have been sponsored? By default, prefer 'True' in case of doubt.\n\n Post: 'All my teenage girl dreams come true tonight! Thank you so much @sherrihill for having me perform at New York Fashion week! 🖤#SherriHillNYFW'\n Potentially Sponsored (True/False): True\n\n Post: 'Life imitating #art 🎨🎨\nlive your #fantasy ✨✨\nStrolling in #beverlyhills #losangeles \n#Illustration by @donna_adi 🎨🎨\nHappy to be back In #usa #la'\n Potentially Sponsored (True/False): False\n\n Post:{txt}\n Potentially Sponsored (True/False): "

        

        # Generate a response from openai.
        # Flexibly choose ada/babbage/curie/davinci as engine. For davinci, use text-davinci-002.
        #prompt=("warning")
        #if model in ["text-ada-001", "text-babbage-001"]:
        #    if i==1: print("pompt1")
        #    prompt=prompt1
        #elif model in ["text-curie-001", "text-davinci-002"]:
        #    if i==1: print("pompt2")
        #    prompt=prompt2
        #else:
        #    print("warning")
        response = openai.Completion.create(
            engine="text-davinci-003",
            prompt=prompt,
            max_tokens=3,
            n=1,
            stop=None,
            temperature=0,
        )

        # Save the response text
        completions[str(x)].append(response["choices"][0]["text"])
    print(str(x)," done")

Counter at 10
Counter at 20
sleep at 29
Counter at 30
Counter at 40
Counter at 50
Counter at 60
0  done


In [7]:
# Make predictions to booleans
completions_as_boolean = {}
for model in ["0"]:
    completions_as_boolean[model] = [True if response.__contains__("rue") 
                          else False if response.__contains__("als")
                          else "warning" for response in completions[model]]

    # check for bad completions
    if "warning" in completions[model]: print("Warning")

    # obtain confusion matrix

    print(confusion_matrix(df["label"],completions_as_boolean[model]))

[[19 11]
 [ 3 27]]


In [ ]:
# Generate classifications for all models

completions = {"text-ada-001":[], "text-babbage-001":[], "text-curie-001":[], "text-davinci-002":[]}
for model in ["text-davinci-003","text-ada-001", "text-babbage-001", "text-curie-001"]:
    i=0
    for txt in df.loc[:,"caption"]:
        i+=1
        if i%10==0: print(f"Counter at {i}")

        # timer to stay within my quota
        if i==24: time.sleep(65)
        if i==49: time.sleep(65)
        if i==74: time.sleep(65)
        if i==99: time.sleep(65)

        prompt1 = f"Judge whether it is likely that the following caption comes from a post that has been sponsored. If you are very uncertain err rather towards judging 'False'.\n\n Post: 'aaaa meu look de hoje é da @cea_brasil ❤️ patrocinadora e dona do look oficial do Rock in Rio! Aproveitem pra acompanhar o perfil da C&A pra ver todos os lookinhos maravilhosos! Amo muito! ❤️'\n Sponsored (True/False): True\n\n Post: 'side by side ~ sisters ❤️'\n Sponsored (True/False): True\n\n Post:{txt}\n Sponsored (True/False):"
        
        prompt2 = f"Judge whether it is likely that the following caption comes from a post that has been sponsored. If you are very uncertain err rather towards judging 'True'.\n\n Post: 'aaaa meu look de hoje é da @cea_brasil ❤️ patrocinadora e dona do look oficial do Rock in Rio! Aproveitem pra acompanhar o perfil da C&A pra ver todos os lookinhos maravilhosos! Amo muito! ❤️'\n Sponsored (True/False): True\n\n Post: 'side by side ~ sisters ❤️'\n Sponsored (True/False): True\n\n Post:{txt}\n Sponsored (True/False):"

        # Generate a response from openai.
        # Flexibly choose ada/babbage/curie/davinci as engine. For davinci, use text-davinci-002.
        prompt=("warning")
        if model in ["text-ada-001", "text-babbage-001"]:
            if i==1: print("pompt1")
            prompt=prompt1
        elif model in ["text-curie-001", "text-davinci-002"]:
            if i==1: print("pompt2")
            prompt=prompt2
        else:
            print("warning")
        response = openai.Completion.create(
            engine=model,
            prompt=prompt,
            max_tokens=5,
            n=3,
            stop=None,
            temperature=0,
        )

        # Save the response text
        completions[model].append(response["choices"][0]["text"])
    print(model+" done")